In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
df = pd.read_csv("filtered_list.csv")
df

,Track URI,Track Name,Artist Name(s),Duration (ms),Popularity,Explicit,User,playlist year,release_year
0,spotify:track:2D2lazsae9o1UoVPUAdxyT,21,Gracie Abrams,185172,31,False,Emily,2023,2020
1,spotify:track:2RvLlxea9woaqb2ZLgAmv7,emma,Casper,195066,40,False,Emily,2023,2023
2,spotify:track:53dtP2iUMvaF28JZcHnFuU,logical,Olivia Rodrigo,231907,71,False,Emily,2023,2023
3,spotify:track:401qSkk4KOeBVld2aiCAoK,Lovers to Strangers,Chance Peña,174086,52,True,Emily,2023,2023
4,spotify:track:2hkcv7xQJgmLvCY3XSixjG,Dussmann,Betterov,217551,45,False,Emily,2023,2022
...,...,...,...,...,...,...,...,...,...
1195,spotify:track:2jdAk8ATWIL3dwT47XpRfu,Slow Dancing in a Burning Room,John Mayer,241946,78,False,William,2025,2006
1196,spotify:track:4QlzkaRHtU8gAdwqjWmO8n,Friday I'm In Love,The Cure,215160,79,False,William,2025,1992
1197,spotify:track:3wlNmUeoHnM1jcOcXxC6qI,Rivendell,Howard Shore,206026,56,False,William,2025,2001
1198,spotify:track:3p428gAKm9dBoMjBVOIvVu,I Can't Hear It Now (from the series Arcane Le...,Arcane;Freya Ridings,161040,64,False,William,2025,2024


In [2]:
# add duration in minutes (float) and as mm:ss
df['Duration_min'] = (df['Duration (ms)'] / 60000).round(2)

def ms_to_mmss(ms):
    sec = int(ms // 1000)
    m = sec // 60
    s = sec % 60
    return f"{m}:{s:02d}"

df['Duration_mmss'] = df['Duration (ms)'].apply(ms_to_mmss)

# display a preview
df[['Track Name', 'Artist Name(s)', 'Duration (ms)', 'Duration_min', 'Duration_mmss']].head()

,Track Name,Artist Name(s),Duration (ms),Duration_min,Duration_mmss
0,21,Gracie Abrams,185172,3.09,3:05
1,emma,Casper,195066,3.25,3:15
2,logical,Olivia Rodrigo,231907,3.87,3:51
3,Lovers to Strangers,Chance Peña,174086,2.90,2:54
4,Dussmann,Betterov,217551,3.63,3:37


In [7]:
# Numeric columns
numeric_cols = df.select_dtypes(include='number').columns.tolist()

# Boolean columns (True / False)
bool_cols = df.select_dtypes(include='bool').columns.tolist()

# Combined selector options
stat_cols = numeric_cols + bool_cols

stat_cols


['Duration (ms)',
 'Popularity',
 'playlist year',
 'release_year',
 'Duration_min',
 'Explicit']

In [10]:
output = widgets.Output()

def user_stats_plot(column):
    output.clear_output(wait=True)

    with output:
        # Boolean column handling
        if column in bool_cols:
            stats = (
                df
                .assign(_bool_as_int=df[column].astype(int))
                .groupby('User')['_bool_as_int']
                .agg(
                    mean='mean',   # proportion explicit
                    sum='sum',     # count explicit
                    min='min',
                    max='max'
                )
            )

            display(stats)
        # Numeric column handling
        else:
            stats = (
                df
                .groupby('User')[column]
                .agg(['mean', 'median', 'min', 'max'])
            )

            display(stats)
        plt.tight_layout()
        plt.show()


In [12]:
column_selector = widgets.Dropdown(
    options=stat_cols,
    description='Column:'
)

column_selector.observe(
    lambda change: user_stats_plot(change['new']),
    names='value'
)

display(column_selector, output)

# Initial render
user_stats_plot(column_selector.value)


Dropdown(description='Column:', options=('Duration (ms)', 'Popularity', 'playlist year', 'release_year', 'Dura…

Output()

In [15]:
from IPython.display import IFrame, display, HTML
import webbrowser

# pick a random track (one representative row)
row = df.sample(1).iloc[0]

track_uri = row['Track URI']
track_name = row['Track Name']
artist = row['Artist Name(s)']

# find all other rows that match on both track name and artist (exclude the sampled row)
matches = df[
    (df['Track Name'] == track_name) &
    (df['Artist Name(s)'] == artist)
].reset_index(drop=True)

# open the track in the browser and embed the Spotify player
webbrowser.open(track_uri)
display(matches)

,Track URI,Track Name,Artist Name(s),Duration (ms),Popularity,Explicit,User,playlist year,release_year,Duration_min,Duration_mmss
0,spotify:track:7lQ8MOhq6IN2w8EYcFNSUk,Without Me,Eminem,290320,88,True,Keke,2024,2002,4.84,4:50
1,spotify:track:7lQ8MOhq6IN2w8EYcFNSUk,Without Me,Eminem,290320,88,True,Keke,2025,2002,4.84,4:50
